# Time Series Gene Clustering Summary

This document contains a summary and current specification of the model we are using for clustering gene expression time series.

## Contents
1. Prior Work
2. Current Model
3. What We're doing
4. Moving Forward

## Prior Work
This will get filled out more in the future, reread papers, and do lit search

- HMM mixture model: Train a mixutre of HMMs, extract clustes by assigning each sequence to a generating HMM
- Hidden state trajectory clustering: train one HMM on all the genes, cluster based on optimal hidden state sequence
- Infinite state HMM clustering

## Data

From the MyConnectome Project, we have RNA-Seq data from 48 time points collected over the course of roughly 18 months. We choose to omit 5 of the samples due to low RIN values.

For the purposes of our model each gene's expression profile is centered and scaled to unit variance. The values observed by our model are a measure of relative over or under expression with respect to an individual genes average epxression level over the course of the study

## Current Model

The basic idea is that given some state space with transitions $\phi$ and emissions distributions $\theta$ we would like to find a group of sequences over the state space generating our observed data. In this way we would like to describe how all of our observed gene sequences were generated from a number of cannonical hidden state trajectories.

Intuitively we want to capture some underlying biological processes and their corresponding gene expression trajectories. Under this model a gene is a participant/member in some unseen process/cluster. So each gene is a noisy observation of that process.

We have observed that if we try to cluster by optimal path each gene tends to go to its own unique optimal path and so we do not achieve a clustering. Identirying a smaller number of underlying hidden state sequences to generate the data is an attempt to resolve the sensitivity of clustering on optimal paths of individual gene sequences.

The model differs from the mixture of HMMs approach in that there is one underlying state set and one transition structure. Perhaps a similar approach could be taken in the mixture of HMMs context by having shared state-space, or doing some sort of shared learning in the transition structure of the mixture components (haven't considered this fully)

The model differs from both state-sequence clustering and HMM mixture clustering in that the way we evaluate the probability of a cluster only counts the transitions once. We are picking our trajectories, and then say that all the genes in the cluster associated with that trajectory are generated by it. Perhaps this is not a good thing-- in clusters with many genes the total liklihood of the cluster will be dominated by the conditional liklihood of the genes sequences given the trajectory.

![model diagram](../figures/fixed_hmm.png)


The dependence $T$ has on $\phi$ has to do with how probable a transition from states $T_t \rightarrow T_{t+1}$ is.

Expanding T and y into the sequences they represent we have:
![epanded model diagram](../figures/fixed_hmm_expand.png)


## What We're Actually Doing

### 1. Train the HMM

We train an N state HMM with univariate Gaussian emissions.

$ M = (\phi, \theta) $ where

Each $ \phi_i,  i = 1, 2, ..., N $   multinomial distribution over states from state i

Each $ \theta_i  \; \tilde \quad N(\mu_i, \sigma^2_i), i = 1, 2, ..., N $ univariate gaussian emissions for state i

We learn:
- transition probabilitites $\phi_{i,j}$
- mean $\mu_i$ and variance $\sigma^2_i$ and variance for eat emissions distribution

In this case the model generates each gene's expression trajectory independently of each other gene. The model parameters here are the ones that maximize the likelihood of generating all of our observed sequences.

This normally goes without saying. But what we are looking for is a model that generates multiple observed sequences as multiple observations of the same underlying process.

Once these parameters are learned they are fixed. **These HMM parameters are not updated at any point during the clustering**. In the final version of this model I don't think it makes sense to keep these parameters fixed. We've just been keeping them fixed for now while we figure out the merging/clustering scheme.


Learned transition matrix we have for a 3 state HMM looks like this. The general trend seems to be "Move towards the middle and stay there"
![3-state transition heatmap](../figures/2017-02-20-kt-3n_transition_heatmap.png)

Densities for the learned distributions. There is a lof of overlap here. How would this look different if we fixed the variances to some relatively small number? Having a lot of overlap in the emissions distributions will cause us to care more about having likely transitions in a hidden state sequence.
![3-state transition heatmap](../figures/2017-02-20-kt-3n-distributions.png)

### 2. Agglomerative Clustering

Our goal here is to pick a set of hidden state trajectories that generate all of our observed data. To do this we are employing an agglomerative clustering scheme on the Viterbi path of each cluster.

#### I. Viterbi Path
For a given observation sequence $y = (y_1, y_2, ..., y_T)$, the Viterbi path is the hidden state sequence $x = (x_1, x_2, ..., x_T)$ maximimzing
$$
P(x, y) = P(x_1)P(x_2 \,|\, x1) ... P(x_T \, |\, x_{T-1})P(y_1 \,|\, x_1) P(y_2 \,|\, x_2) ... P(y_T \,|\, x_T) \quad (1)
$$

#### II. Viterbi Path with multiple observations
**This requires attention, not sure if this is a good way to go about what we want.**

So our HMM $M$ generates individual gene expression sequences, but we are trying to use it to find groups of genes generated by the hidden state trajectory. Consider a group of observed sequences $S = \{y^{(1)}, y^{(2)}, ..., y^{(k)}\}$ We've been computing the probability of a hidden state sequence $x$ as:
$$
P(S, x) = P(x_1)P(x_2 \,|\, x1) ... P(x_T \, |\, x_{T-1})  \prod_{t=1}^{T}\prod_{i=1}^{k} P(y_{t}^{(i)}\, |\, x_t) \quad (2)
$$

Take note that the transitions are only being counted once.

If we find the path maximizing $P(S, x)$ as described above what we are essentially doing is finding the Viterbi path for S over a new model $M_k = (\phi_k, \theta_k)$, where 
$$
\phi_k = \phi \\
\theta_{k, i} \, \tilde \quad Guassian([\mu_i, \mu_i, \dots, \mu_i], \, \Sigma ) \; \Sigma = 
\begin{bmatrix}
    \sigma_i^2 & 0 & 0 & \dots  & 0 \\
    0 & \sigma_i^2 & 0 & \dots  & 0 \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & 0 & \dots  & \sigma_i^2
\end{bmatrix}
\quad (3)
$$

Where $\mu_i$ and $\sigma_i^2$ correspond to the parameters of the emissions distribution $\theta_i$ in $M$.

$P(S_t \,|\, x_t, M_k) \, = \, \prod_{i=1}^{k} P(y_{t}^{(i)}\, |\, x_t) \;$ for some $S$ at time $t$ 

As Alexis mentioned, the transition structure $\phi$ learned in $M$ isn't necessarily optimal in this new context where we are trying to describe the generation of our observed set of sequences $S$

### III. Clustering

#### a. Viterbi Path

The agglomerative clustering over Viterbi paths has been performed as followed. At each step we consider all possible merges between existing clusters. We select the merge that is the least costly to the liklihood of the clustering overall.

$D = \{y^{(i)}\}_{1 \leq i \leq G} \;$ be our observed data, where there are $G$ genes.

We initialize with each gene sequence is in its own singleton cluster $S_i = \{y^{(i)}\}, i = 1, 2, \dots, G$.

Denote the clustering by $C = \{ S_1, S_2, \dots, S_G\}$


**While there is more than one cluster:**
1. Compute $T_{S_i \cup S_j}$, the path $x$ maximizing $P(S_i \cup S_j, \,x)$  for all $(S_i, S_j) \in C \times C$
2. Select $ S_u, S_v = argmax_{S_i, S_j} \; logP(S_i \cup S_j, T_{S_i,S_j})  - (logP(S_i \, | \, T_{S_i}) + logP(S_j \, | \, T_{S_j}))  \\$
3. $C \cup \{S_u \cup S_v\} \setminus\{S_u, S_v\} $

**Depending on the rule we come up with for cutting the tree we'll probably be able to terminate the clustering part sooner than before we build to whole tree. That is still in development.**

![vit merge liklihood](../figures/2017-02-20-kt-vit-1500-total.png)

Here is an example of what this clustering scheme looks like on a random 1500 genes from the MyConnectome dataset. The liklihood of the clustering here is evaluated as the sum of log liklihoods of the best paths for each cluster. 

One way we could consider cutting the tree is simply cutting where the liklihood of the data is maximized under our modelling assumption. We get our highest liklihood after merge 1181. Cuttin the tree here gives us a little more that 300 clusters for 1500 genes.

How will this work with larger sets of genes? Where is the clustering's viterbi path liklihood maximized for G genes? We need to find this out if we are going to come up with a sensible policy for picking a place to cut the tree into clusters and wheter or not picking the best total viterbi path liklihood across all cluster merges is the best way to pick our clustering.

![vit merge liklihood](../figures/2017-02-20-kt-vit-max-prob-cluster-sizes.png)

Here is a look at the distribution over cluster sizes. Most of our clusters are pretty small. If we want to extract larger clusters we still need to come up with some critereon for cutting the tree higher.

#### b. Marginalize out state sequences

Something we were considering earlier was not looking at a particular generating state sequence but simply that all members in a cluster were generated from some identicle state sequence.

If I am not mistaken, when we are only counting the transition probabilities once, this task is the same as finding some clustering/partitioning of the data $C = \{ S_1, S_2, \dots, S_G=k\}$ maximizing $\prod_{i=1}^{k} P(S_i \,|\, M_{c_i})$, where $c_i = |S_i|$ and $M_{c_i}$ is an HMM with identicle transition structure to M but draws from the emissions distribution $c_i$ times at each step (as described above). So we have

$D = \{y^{(i)}\}_{1 \leq i \leq G} \;$ be our observed data, where there are $G$ genes.

We initialize with each gene sequence is in its own singleton cluster $S_i = \{y^{(i)}\}, i = 1, 2, \dots, G$.

Denote the clustering by $C = \{ S_1, S_2, \dots, S_G\}$


**While there is more than one cluster:**
1. Compute $P_{i,j} = P(S_i \cup S_j, \ | \; M_{c_{i} + c_{j}})$ where $c_{i} = |S_i|$
2. Select $ u, v = argmax_{i, j}\; logP_{i,j} - (logP(S_i \; | \; M_{c_i}) + logP(S_j \; | \; M_{c_j})) $
3. $C \cup \{S_u \cup S_v\} \setminus\{S_u, S_v\} $

![1500 merge liklihood](../figures/2017-02-20-kt-1500-total.png)

If we do this we see that the liklihood of the data under the sequential clusterings is maximized around the 400th merge, giving us around 1100 clusters with 1 or 2 genes per cluster. Clearly this isn't very informative so we'd have to come up with some way of determining a cutoff higher up the tree.

### 3. K-path-clustering

It is an attractive feature of a clustering algorithm to not have to choose k. However, if in the case of the agglomerative clustering described above we are interested extracting larger clusters then we will at least in some indirect way be choosing k.

While we are at that, we could just as well skip the agglomerative bit of things and pick a k by normal means. The greedy agglomerative approach may give us some k-clustering but it is not necessarily an optimal k-clustering. We can explore methods for finding good k clusterings.

The obvious approach here (noting that we are still working with fixed HMM parameters) is to perform hard EM. Starting with some initial k clustering of the data we iteratively compute the optimal path for each cluster $\{ T_1, \dots T_k\}$ (M-step) and reassign sequences to the path best describing it (E-step).

Note that in the count-the-transitions-once approach the E-step boils down to computing the conditional probability $P(y \,|\, x) = P(y_1 \,|\, x_1) P(y_2 \,|\, x_2) ... P(y_T \,|\, x_T) \quad x \in \{ T_1, \dots T_k\}$

So while the hidden state trajectories are computed with regard for transition probabilities, the assignment of a gene sequence to a given trajectory is done based on the conditional probability that the such a hidden state sequence has been generated

We are only counting the transitions once, and trajectories $\{ T_1, \dots T_k\}$ for the k clusters are fixed when we are reassigning sequences to clusters. Then, the optimal hard assignment of a sequence to a cluster is the one minimizing the conditional probability of the observed sequence on the trajectory generating that cluster.



## Thoughts, Reflections, & Moving Forward

How sensible is it to keep the learned HMM parameters fixed throughout the clustering process?
How bad of an assumption is it to say these parameters are fixed? How much can we expect the model to improve by reistimating transition and/or emissions distributions.

Ultimately I think this model will be include estimating transition and/or emissions parameters. I feel like the only reason we are keeping them fixed for now is that 1) the idea for this model came out of working in the HMM state-sequence clustering context, and 2) so that we can focus on figuring out what the rest of the model is doing in terms of how trajectories are generated and how gene expression sequences are generated from those trajectories.

Now, I think we should also go back and look at what the emissions distributions are exactly. In the HMM they are just univariate Normal. However, the $\theta$ written in the diagram is not this. The $\theta$ in the model supports multiple observations at once. Replicating the HMM to produce differently dimensioned outputs seems kind of hacky, and as Alexis mentions the parameters learned for the univariate case are not necessarily optimal in the higher dimensional cases.

Is there a way to have a multivariate normal distribution of random dimension? What should the covariance matrix look like? In the above we observe each data point in a cluster at a given time point as an independent observation of a state in that trajectory. This area needs more thought.

Supposing we had randomly dimensioned multivariate normals as our emissions distribution, how do we paramaterize the distribution over this dimension. This would also essentially be a distribution over cluster sizes.
